#### Helper Functions

In [66]:
import yaml
import sympy as sp

def generate_aggregation_yaml_from_file(config_file, parameters):
    with open(config_file, "r") as file:
        template = file.read()
    
    yaml_text = template.format(**parameters)
    dict_obj = yaml.safe_load(yaml_text)

    fname_prefix = dict_obj.get("metadata").get("name")

    assert fname_prefix is not None, "fname not in correct position???"
    return fname_prefix, yaml_text


In [7]:
def write_yaml_to_file(fname_prefix, yaml_text):
    with open(f"{fname_prefix}.yaml", "w") as f:
        f.write(yaml_text)

In [59]:
def fname_to_redis_key(fname_prefix):
    '''
        Example Input: "index.multiple.spot.multiple.public.FYN.USD.delta.affyn-mm" 
        Output:"delta.multiple.spot.multiple.public.FYN.USD.affyn-mm"
    '''
    string_lst = fname_prefix.split(".")
    string_lst.pop(0)
    temp = string_lst.pop(-2)
    string_lst = [temp] + string_lst
    return '.'.join(string_lst) 

#### Volume (PES)

In [63]:
# Example usage: Volume
config_file = "./configs_templates/volume_template.yaml"
parameters = {
    "exchange": "bitmart",
    "exchangeType": "spot",
    "exchangeProtocol": "ws",
    "currency1": "IXS",
    "currency2": "USDT",
    "clientName": "ixswap",
    "accountID": "bitmart-spot-0",
    "usdtIndex": "tick.bitmart.spot.ws.public.IXS.USDT.USDT", # this can be either basket index or the exchange tick
    "tickPublicOrderbook": "tick.bitmart.spot.ws.public.IXS.USDT.USDT"
}

fname_prefix, yaml_text = generate_aggregation_yaml_from_file(config_file, parameters)
# write_to_file(fname_prefix, yaml_text)
print(fname_prefix, yaml_text)

volume.bitmart.spot.IXS.USDT.ixswap.bitmart-spot-0 apiVersion: microservice.flowdesk.co/v1alpha1
kind: Volume
metadata:
  labels:
    app.kubernetes.io/managed-by: flux
    app.kubernetes.io/created-by: flux
  name: volume.bitmart.spot.IXS.USDT.ixswap.bitmart-spot-0
  namespace: ixswap
spec:
  deploymentSpec:
    projectId: infrastructure-prod-92821
    imageName: asia-northeast1-docker.pkg.dev/infrastructure-prod-92821/main-asia/flowdeskmarkets/volume
    imageTag: v3.4.0
  settings:
    accountID: bitmart-spot-0
    authType: private
    clientName: ixswap
    currency1: IXS
    currency2: USDT
    dynamicSettings:
      V0: 3800
      baseProbabilityThreshold: 0.0015
      digitsPrice: 5
      digitsVolume: 2
      directionalityFactor: 0.65
      minStepVolume: 0
      minTradeAmount: 200
      preventSpreadCrossing: true
      priceNoiseRange: 0.005
      priceStep: 100
      priceVolatilityFactor: 1
      pushToExecutor: false
      serviceOn: false
    exchange: bitmart
    exch

In [64]:
write_yaml_to_file(f"configs/{fname_prefix}", yaml_text)

#### PATH const definitions

In [9]:
TEMPLATE_BASE_PATH = "./configs_templates/" ### change this line ###

PRIVATE_TEMPLATE_FPATH = TEMPLATE_BASE_PATH + "/private_base_config.yaml"
PUBLIC_TEMPLATE_PATH = TEMPLATE_BASE_PATH + "public_base_config.yaml"
INDEX_TEMPLATE_PATH = TEMPLATE_BASE_PATH + "indexes/"


#### Private (trades, balances, ob)

##### Private Trades

In [10]:
def private_trades_agg(clientName, exchange, exchangeType, exchangeProtocol, currency1, currency2, indexkeyName, usdIndexKeyName, accountID = None, period = None, config_fpath=PRIVATE_TEMPLATE_FPATH):
    '''
    extra fields to add to base aggregation config
    - indexkeyName: global USDT basket Index
    - usdIndexKeyName: global USD basket Index
    - fetchFull: True by default (?)

    Optional Params:
    - accountID: since usually is {exchange}-{exchangeType}-0
    - period: usually is 5000 mseconds
    '''
    params = {
        "clientName": clientName,
        "exchange": exchange,
        "exchangeType": exchangeType,
        "exchangeProtocol": exchangeProtocol,
        "currency1": currency1,
        "currency2": currency2,
        "accountID": f"{exchange}-{exchangeType}-0" if accountID is None else accountID,
        "period": 5000 if period is None else period,
        "marketData": "trades", ## fixed
    }

    fname_prefix, yaml_text = generate_aggregation_yaml_from_file(config_file=config_fpath, parameters=params)

    ## add extra fields
    temp = yaml.safe_load(yaml_text)
    temp["spec"]["settings"]["indexKeyName"] = indexkeyName ## usdt basket
    temp["spec"]["settings"]["usdIndexKeyName"] = usdIndexKeyName ## ## usd basket
    temp["spec"]["settings"]["fetchFull"] = True

    yaml_text = yaml.dump(temp)

    return fname_prefix, yaml_text


In [11]:
fname_prefix, yaml_text = private_trades_agg(
    clientName="affyn",
    exchange="gate",
    exchangeType="spot",
    exchangeProtocol="ws",
    currency1='ETH',
    currency2="USDT",
    indexkeyName="basket.multiple.spot.multiple.public.ETH.USDT.asia.global",
    usdIndexKeyName="basket.multiple.spot.multiple.public.ETH.USD.asia.global"
)

In [12]:
print(yaml_text)

apiVersion: microservice.flowdesk.co/v1alpha1
kind: Aggregation
metadata:
  labels:
    app.kubernetes.io/created-by: flux
    app.kubernetes.io/managed-by: flux
  name: aggregation.gate.spot.ws.private.ETH.USDT.trades.affyn.gate-spot-0
  namespace: affyn
spec:
  deploymentSpec:
    imageName: asia-northeast1-docker.pkg.dev/infrastructure-prod-92821/main-asia/flowdeskmarkets/aggregation
    imageTag: v2.188.4
    injectorImageName: asia-northeast1-docker.pkg.dev/infrastructure-prod-92821/main-asia/gcp-secretmanager-apikeys-injector
    injectorImageTag: v1.0.1
    projectId: infrastructure-prod-92821
  settings:
    accountID: gate-spot-0
    authType: private
    clientName: affyn
    currency1: ETH
    currency2: USDT
    exchange: gate
    exchangeType: spot
    fetchFull: true
    indexKeyName: basket.multiple.spot.multiple.public.ETH.USDT.asia.global
    marketData: trades
    period: 5000
    protocol: ws
    usdIndexKeyName: basket.multiple.spot.multiple.public.ETH.USD.asia.glob

##### Private Balance, OrderBook

In [13]:
def private_orderbook_agg(clientName, exchange, exchangeType, exchangeProtocol, currency1, currency2, accountID=None, period=None, config_fpath=PRIVATE_TEMPLATE_FPATH):
    params = {
        "clientName": clientName,
        "exchange": exchange,
        "exchangeType": exchangeType,
        "exchangeProtocol": exchangeProtocol,
        "currency1": currency1,
        "currency2": currency2,
        "accountID": f"{exchange}-{exchangeType}-0" if accountID is None else accountID,
        "period": 20000 if period is None else period,
        "marketData": "orderbook", ## fixed
    }
    # returns fname_prefix, yaml_text
    return generate_aggregation_yaml_from_file(config_file=config_fpath, parameters=params)


In [14]:
fname_prefix, yaml_text = private_orderbook_agg(
    clientName="affyn",
    exchange="gate",
    exchangeType="spot",
    exchangeProtocol="ws",
    currency1='ETH',
    currency2="USDT",
)

In [15]:
print(yaml_text)

# aggregation_base_config.yaml
apiVersion: microservice.flowdesk.co/v1alpha1
kind: Aggregation
metadata:
  labels:
    app.kubernetes.io/managed-by: flux
    app.kubernetes.io/created-by: flux
  name: aggregation.gate.spot.ws.private.ETH.USDT.orderbook.affyn.gate-spot-0
  namespace: affyn # if public then should be global
spec:
  deploymentSpec:
    projectId: infrastructure-prod-92821
    imageName: asia-northeast1-docker.pkg.dev/infrastructure-prod-92821/main-asia/flowdeskmarkets/aggregation
    imageTag: v2.188.4
    injectorImageName: asia-northeast1-docker.pkg.dev/infrastructure-prod-92821/main-asia/gcp-secretmanager-apikeys-injector
    injectorImageTag: v1.0.1
  settings:
    clientName: affyn # e.g. stader
    accountID: gate-spot-0 # e.g. okex-spot-0
    exchange: gate # e.g. okex
    exchangeType: spot # spot, futures, options
    protocol: ws # rest, ws, fix(?)
    authType: private
    marketData: orderbook # balance, orderbook, trades(specical case)
    currency1: ETH
    

In [16]:
def private_balance_agg(clientName, exchange, exchangeType, exchangeProtocol, currency1, currency2, accountID=None, period=None, config_fpath=PRIVATE_TEMPLATE_FPATH):
    params = {
        "clientName": clientName,
        "exchange": exchange,
        "exchangeType": exchangeType,
        "exchangeProtocol": exchangeProtocol, ## expected most are rest
        "currency1": currency1,
        "currency2": currency2,
        "accountID": f"{exchange}-{exchangeType}-0" if accountID is None else accountID,
        "period": 10000 if period is None else period,
        "marketData": "balance", ## fixed
    }
    # returns fname_prefix, yaml_text
    return generate_aggregation_yaml_from_file(config_file=config_fpath, parameters=params)


In [17]:
fname_prefix, yaml_text = private_balance_agg(
    clientName="affyn",
    exchange="gate",
    exchangeType="spot",
    exchangeProtocol="rest",
    currency1='ETH',
    currency2="USDT",
)

In [18]:
print(yaml_text)

# aggregation_base_config.yaml
apiVersion: microservice.flowdesk.co/v1alpha1
kind: Aggregation
metadata:
  labels:
    app.kubernetes.io/managed-by: flux
    app.kubernetes.io/created-by: flux
  name: aggregation.gate.spot.rest.private.ETH.USDT.balance.affyn.gate-spot-0
  namespace: affyn # if public then should be global
spec:
  deploymentSpec:
    projectId: infrastructure-prod-92821
    imageName: asia-northeast1-docker.pkg.dev/infrastructure-prod-92821/main-asia/flowdeskmarkets/aggregation
    imageTag: v2.188.4
    injectorImageName: asia-northeast1-docker.pkg.dev/infrastructure-prod-92821/main-asia/gcp-secretmanager-apikeys-injector
    injectorImageTag: v1.0.1
  settings:
    clientName: affyn # e.g. stader
    accountID: gate-spot-0 # e.g. okex-spot-0
    exchange: gate # e.g. okex
    exchangeType: spot # spot, futures, options
    protocol: rest # rest, ws, fix(?)
    authType: private
    marketData: balance # balance, orderbook, trades(specical case)
    currency1: ETH
    

#### Public (tick, trades, ob)

##### Public tick

In [19]:
def public_tick_agg(exchange, exchangeType, exchangeProtocol, currency1, currency2, effectiveCurrencies2=None, config_fpath=PUBLIC_TEMPLATE_PATH):
    params = {
        "exchange": exchange,
        "exchangeType": exchangeType,
        "exchangeProtocol": exchangeProtocol,
        "currency1": currency1,
        "currency2": currency2,
        "authType": "public", ## fixed
        "marketData": "tick", ## fixed
    }

    fname_prefix, yaml_text = generate_aggregation_yaml_from_file(config_file=config_fpath, parameters=params)

    ## add / pop extra fields
    temp = yaml.safe_load(yaml_text)

    ## add effective currency
    if effectiveCurrencies2 is None:
        effectiveCurrencies2 = 'USD'
        indexName = f"basket.multiple.spot.multiple.public.{currency2}.USD.asia.global"
    else:
        indexName = f"basket.multiple.spot.multiple.public.{currency2}.{effectiveCurrencies2}.asia.global"

    ## TODO: check in redis if such key (indexName) exists
    
    temp["spec"]["settings"]["effectiveCurrencies2"] = [
        {
            "name": effectiveCurrencies2,
            "indexName": indexName
        }
    ]

    yaml_text = yaml.dump(temp)

    return fname_prefix, yaml_text

In [20]:
fname_prefix, yaml_text = public_tick_agg(
    exchange="gate",
    exchangeType="spot",
    exchangeProtocol="ws",
    currency1='ETH',
    currency2="USDT",
)

In [21]:
print(yaml_text)

apiVersion: microservice.flowdesk.co/v1alpha1
kind: Aggregation
metadata:
  labels:
    app.kubernetes.io/created-by: flux
    app.kubernetes.io/managed-by: flux
  name: aggregation.gate.spot.ws.public.ETH.USDT.tick
  namespace: global
spec:
  deploymentSpec:
    imageName: asia-northeast1-docker.pkg.dev/infrastructure-prod-92821/main-asia/flowdeskmarkets/aggregation
    imageTag: v2.188.4
    injectorImageName: asia-northeast1-docker.pkg.dev/infrastructure-prod-92821/main-asia/gcp-secretmanager-apikeys-injector
    injectorImageTag: v1.0.1
    projectId: infrastructure-prod-92821
  settings:
    authType: public
    currency1: ETH
    currency2: USDT
    effectiveCurrencies2:
    - indexName: basket.multiple.spot.multiple.public.USDT.USD.asia.global
      name: USD
    exchange: gate
    exchangeType: spot
    marketData: tick
    protocol: ws



##### Public trades

In [22]:
def public_trades_agg(exchange, exchangeType, exchangeProtocol, currency1, currency2, config_fpath=PUBLIC_TEMPLATE_PATH):
    params = {
        "exchange": exchange,
        "exchangeType": exchangeType,
        "exchangeProtocol": exchangeProtocol,
        "currency1": currency1,
        "currency2": currency2,
        "authType": "public", ## fixed
        "marketData": "trades", ## fixed
    }

    fname_prefix, yaml_text = generate_aggregation_yaml_from_file(config_file=config_fpath, parameters=params)

    ## add / pop extra fields
    temp = yaml.safe_load(yaml_text)

    yaml_text = yaml.dump(temp)

    return fname_prefix, yaml_text

In [23]:
fname_prefix, yaml_text = public_trades_agg(
    exchange="gate",
    exchangeType="spot",
    exchangeProtocol="ws",
    currency1='ETH',
    currency2="USDT",
)

In [24]:
print(yaml_text)

apiVersion: microservice.flowdesk.co/v1alpha1
kind: Aggregation
metadata:
  labels:
    app.kubernetes.io/created-by: flux
    app.kubernetes.io/managed-by: flux
  name: aggregation.gate.spot.ws.public.ETH.USDT.trades
  namespace: global
spec:
  deploymentSpec:
    imageName: asia-northeast1-docker.pkg.dev/infrastructure-prod-92821/main-asia/flowdeskmarkets/aggregation
    imageTag: v2.188.4
    injectorImageName: asia-northeast1-docker.pkg.dev/infrastructure-prod-92821/main-asia/gcp-secretmanager-apikeys-injector
    injectorImageTag: v1.0.1
    projectId: infrastructure-prod-92821
  settings:
    authType: public
    currency1: ETH
    currency2: USDT
    exchange: gate
    exchangeType: spot
    marketData: trades
    protocol: ws



##### Public OB

In [25]:
def public_orderbook_agg(exchange, exchangeType, exchangeProtocol, currency1, currency2, config_fpath=PUBLIC_TEMPLATE_PATH):
    """
    Generates a public order book aggregation YAML file based on the provided exchange, exchange type, exchange protocol, and currency pairs.

    This function constructs a dictionary of parameters, including exchange details and currency pairs, and uses these to generate a YAML file.  
    It then loads the YAML file, optionally modifies it by adding or removing fields, and finally returns the filename prefix and the modified YAML text.

    Parameters:
        exchange (str): The name of the exchange (e.g., binance, okex)
        exchangeType (str): The type of the exchange (e.g., spot, futures).
        exchangeProtocol (str): The protocol used by the exchange (e.g., rest, ws, fix).
        currency1 (str): The first currency in the pair, assumed all caps (e.g. BTC)
        currency2 (str): The second currency in the pair, assumed all caps (e.g. USDT)
        config_fpath (str, optional): The path to the configuration file. Defaults to PUBLIC_TEMPLATE_PATH.

    Returns:
        tuple: A tuple containing the filename prefix and the YAML text.

    Note:
        The market data is fixed to "orderbook" for this function.
    """
    params = {
        "exchange": exchange,
        "exchangeType": exchangeType,
        "exchangeProtocol": exchangeProtocol,
        "currency1": currency1,
        "currency2": currency2,
        "marketData": "orderbook", ## fixed
    }

    fname_prefix, yaml_text = generate_aggregation_yaml_from_file(config_file=config_fpath, parameters=params)

    ## add / pop extra fields
    temp = yaml.safe_load(yaml_text)

    yaml_text = yaml.dump(temp)

    return fname_prefix, yaml_text


In [26]:
fname_prefix, yaml_text = public_orderbook_agg(
    exchange="gate",
    exchangeType="spot",
    exchangeProtocol="ws",
    currency1='ETH',
    currency2="USDT",
)

In [27]:
print(yaml_text)

apiVersion: microservice.flowdesk.co/v1alpha1
kind: Aggregation
metadata:
  labels:
    app.kubernetes.io/created-by: flux
    app.kubernetes.io/managed-by: flux
  name: aggregation.gate.spot.ws.public.ETH.USDT.orderbook
  namespace: global
spec:
  deploymentSpec:
    imageName: asia-northeast1-docker.pkg.dev/infrastructure-prod-92821/main-asia/flowdeskmarkets/aggregation
    imageTag: v2.188.4
    injectorImageName: asia-northeast1-docker.pkg.dev/infrastructure-prod-92821/main-asia/gcp-secretmanager-apikeys-injector
    injectorImageTag: v1.0.1
    projectId: infrastructure-prod-92821
  settings:
    authType: public
    currency1: ETH
    currency2: USDT
    exchange: gate
    exchangeType: spot
    marketData: orderbook
    protocol: ws



#### Indexes (delta, basket index, price index, pos factor, vol index)

##### Delta

In [60]:
def index_delta(clientName, currency1, effectiveCurrencies2, tradeAggList, deltaCurrency=None, config_fpath = INDEX_TEMPLATE_PATH+"delta_template.yaml"):
    if deltaCurrency is None:
        deltaCurrency = currency1
    else:
        assert deltaCurrency in [currency1, effectiveCurrencies2], "deltaCurrency has to be either currency1 or effectiveCurrencies2"
    
    params = {
        "currency1": currency1,
        "effectiveCurrencies2": effectiveCurrencies2,
        "clientName": clientName,
        "deltaCurrency": deltaCurrency
    }

    fname_prefix, yaml_text = generate_aggregation_yaml_from_file(config_file=config_fpath, parameters=params)

    temp = yaml.safe_load(yaml_text)

    ## add trade aggres to compose delta
    temp["spec"]["settings"]["components"] = tradeAggList

    yaml_text = yaml.dump(temp)

    return fname_prefix, yaml_text


In [61]:
fname_prefix, yaml_text = index_delta(
    clientName="affyn",
    currency1='FYN',
    effectiveCurrencies2="USD",
    tradeAggList=[
        "trades.uniswapv3-polygon.spot.rest.private.FYN.ETH.affyn.uniswapv3-polygon-spot-0",
        "trades.gate.spot.ws.private.FYN.USDT.affyn.gate-spot-0",
        "trades.mexc.spot.ws.private.FYN.USDT.affyn.mexc-spot-0",
        ]
)

In [31]:
print(yaml_text)

apiVersion: microservice.flowdesk.co/v1alpha1
kind: Index
metadata:
  labels:
    app.kubernetes.io/created-by: flux
    app.kubernetes.io/managed-by: flux
  name: index.multiple.spot.multiple.public.FYN.USD.delta.affyn-mm
  namespace: global
spec:
  deploymentSpec:
    imageName: asia-northeast1-docker.pkg.dev/infrastructure-prod-92821/main-asia/flowdeskmarkets/index
    imageTag: v1.15.1
    projectId: infrastructure-prod-92821
  settings:
    authType: public
    components:
    - trades.uniswapv3-polygon.spot.rest.private.FYN.ETH.affyn.uniswapv3-polygon-spot-0
    - trades.gate.spot.ws.private.FYN.USDT.affyn.gate-spot-0
    - trades.mexc.spot.ws.private.FYN.USDT.affyn.mexc-spot-0
    currency1: FYN
    currency2: USD
    customName: affyn-mm
    disableExpiry: true
    exchange: multiple
    exchangeType: spot
    indexSettings:
      deltaCurrency: FYN
      sourceFilters:
      - market-making
      - hedging
      - unmapped
      - flowdeskAPI
      - brokerage
    marketData: 

##### PosFactor

In [38]:
def index_position_factor(currency1, effectiveCurrencies2, deltaIndexPrefix, denominator=10000, posfLimit=0.2, config_fpath = INDEX_TEMPLATE_PATH+"position_factor_template.yaml"):
    '''
    deltaIndexPrefix can be list or str
    '''

    if isinstance(deltaIndexPrefix, list):
        deltaIndexPrefix_list = deltaIndexPrefix
        deltaIndexPrefix = False # stuff dummy value to fmt first, then overwrite with list replacement

    params = {
        "currency1": currency1,
        "effectiveCurrencies2": effectiveCurrencies2,
        "deltaIndexPrefix": deltaIndexPrefix,
        "denominator": denominator,
        "posfLimit": posfLimit
    }

    fname_prefix, yaml_text = generate_aggregation_yaml_from_file(config_file=config_fpath, parameters=params)

    if deltaIndexPrefix == False: ## overwrite here
        temp = yaml.safe_load(yaml_text)

        ## add trade aggres to compose delta
        temp["spec"]["settings"]["components"] = deltaIndexPrefix_list

        yaml_text = yaml.dump(temp)

    return fname_prefix, yaml_text


In [44]:
fname_prefix, yaml_text = index_position_factor(
    currency1="FYN",
    effectiveCurrencies2="USD",
    deltaIndexPrefix=fname_to_redis_key("index.multiple.spot.multiple.public.FYN.USD.delta.affyn-mm") # ['delta.multiple.spot.multiple.public.FYN.USD.affyn-mm', "hehe"]
)

In [45]:
print(yaml_text)

apiVersion: microservice.flowdesk.co/v1alpha1
kind: Index
metadata:
  labels:
    app.kubernetes.io/managed-by: flux
    app.kubernetes.io/created-by: flux
  name: index.multiple.spot.ws.private.FYN.USD.basket.position-factor-mm
  namespace: global
spec:
  deploymentSpec:
    projectId: infrastructure-prod-92821
    imageName: asia-northeast1-docker.pkg.dev/infrastructure-prod-92821/main-asia/flowdeskmarkets/index
    imageTag: v1.15.1
  settings:
    customName: position-factor-mm
    exchange: multiple
    exchangeType: spot
    protocol: ws
    authType: private
    marketData: basket
    currency1: FYN
    currency2: USD
    components:
    - index.multiple.spot.multiple.public.FYN.USD.delta.affyn-mm
    indexSettings:
      formula: min(max((a)/10000,-0.2),0.2)
    dataComponent: mid
    minPeriod: 1000
    minVariationThreshold: 0.01
    disableExpiry: true



##### Basket Index (Weighted) USD +> USDT

In [106]:
def index_weighted_USD(currency1, effectiveCurrencies2, ticksList, tickWeights = None, exchangeType = "spot", config_fpath = INDEX_TEMPLATE_PATH+"usd_basket_index_template.yaml"):
    if isinstance(ticksList, str):
        ticksList = [ticksList]
        assert tickWeights in (1, None), "If only one tick then weighting in index must be 1"
        ## no need to define tickweight if only one tick

    else: ## when ticklist is an actual list
        if tickWeights is None: ## edge case handling if weights are not specified
            if (num_tick := len(ticksList)) > 1:
                tickWeights = [round(1/num_tick, ndigits=3) for _ in range(num_tick)]
            else:  
                tickWeights = 1

    params = {
        "currency1": currency1,
        "effectiveCurrencies2": effectiveCurrencies2,
        "exchangeType": exchangeType,
    }

    fname_prefix, yaml_text = generate_aggregation_yaml_from_file(config_file=config_fpath, parameters=params)

    ## overwrite here
    temp = yaml.safe_load(yaml_text)

    tick_names_list = [] # to be used in formula
    for component in ticksList:
        tickName = f"{currency1}{component.split('.')[1]}" ## e.g. {SD}{huobi}
        temp["spec"]["settings"]["components"].append(
            {
                "name": component,
                "alias": tickName,
                "expireAfter": 60000,
                "latency": {
                    "current": 100,
                    "tolerance": 500,
                    "boostFactor": 1
                }
            }
        )
        tick_names_list.append(tickName)
        
    if tickWeights != 1 and tickWeights is not None:
        formula = "+".join([f"Weight({tickName}, {tickWeight})" for tickName, tickWeight in zip(tick_names_list, tickWeights)])
        temp["spec"]["settings"]["indexSettings"]["formula"] = f"Basket({formula})"

    else:
        temp["spec"]["settings"]["indexSettings"]["formula"] = f"Basket(Weight({tick_names_list[0]}, 1))"

    yaml_text = yaml.dump(temp)

    return fname_prefix, yaml_text


In [107]:
fname_prefix , yaml_text = index_weighted_USD(
    currency1="SD", 
    effectiveCurrencies2="USD",
    ticksList=["tick.huobi.spot.ws.public.SD.USDT.USD", "tick.okex.spot.ws.public.SD.USDT.USD", "tick.gate.spot.ws.public.SD.USDT.USD"]  #"tick.huobi.spot.ws.public.SD.USDT.USD"
)

In [103]:
print(yaml_text)

apiVersion: microservice.flowdesk.co/v1alpha1
kind: BasketIndex
metadata:
  labels:
    app.kubernetes.io/created-by: flux
    app.kubernetes.io/managed-by: flux
  name: index.multiple.spot.multiple.public.sd.usd.basket.asia.global
  namespace: global
spec:
  deploymentSpec:
    imageName: asia-northeast1-docker.pkg.dev/infrastructure-prod-92821/main-asia/flowdeskmarkets/basket-index
    imageTag: v1.9.0
    nodeAffinity:
    - workload-performance
    projectId: infrastructure-prod-92821
  settings:
    authType: public
    components:
    - alias: SDhuobi
      expireAfter: 60000
      latency:
        boostFactor: 1
        current: 100
        tolerance: 500
      name: tick.huobi.spot.ws.public.SD.USDT.USD
    - alias: SDokex
      expireAfter: 60000
      latency:
        boostFactor: 1
        current: 100
        tolerance: 500
      name: tick.okex.spot.ws.public.SD.USDT.USD
    - alias: SDgate
      expireAfter: 60000
      latency:
        boostFactor: 1
        current: 100

In [104]:
def index_weighted_USDT(currency1, currency2, effectiveCurrencies2, usdIndexKey, exchangeType="spot", config_fpath = INDEX_TEMPLATE_PATH+"usdt_index_template.yaml"):
    params = {
        "currency1": currency1,
        "currency2": currency2,
        "exchangeType": exchangeType
    }

    fname_prefix, yaml_text = generate_aggregation_yaml_from_file(config_file=config_fpath, parameters=params)
    
    ## overwrite here
    temp = yaml.safe_load(yaml_text)
    temp["spec"]["settings"]["components"] = [
        {
            "name": usdIndexKey,
            "alias": currency1,
            "expireAfter": 60000,
            "latency":{
                "current": 150,
                "tolerance": 500,
                "boostFactor": 1
            }
        },
        {
            "name": f"basket.multiple.spot.multiple.public.{currency2}.{effectiveCurrencies2}.asia.global",
            "alias": currency2,
            "expireAfter": 60000,
            "latency":{
                "current": 150,
                "tolerance": 500,
                "boostFactor": 1
            }
        },
    ]
    yaml_text = yaml.dump(temp)

    return fname_prefix, yaml_text

In [108]:
fname_prefix , yaml_text = index_weighted_USDT(
    currency1="SD", 
    effectiveCurrencies2="USD",
    currency2 = "USDT",
    usdIndexKey = fname_to_redis_key(fname_prefix)
)

In [109]:
print(yaml_text)

apiVersion: microservice.flowdesk.co/v1alpha1
kind: BasketIndex
metadata:
  labels:
    app.kubernetes.io/created-by: flux
    app.kubernetes.io/managed-by: flux
  name: index.multiple.spot.multiple.public.SD.USDT.basket.asia.global
  namespace: global
spec:
  deploymentSpec:
    imageName: asia-northeast1-docker.pkg.dev/infrastructure-prod-92821/main-asia/flowdeskmarkets/basket-index
    imageTag: v1.9.0
    nodeAffinity:
    - workload-performance
    projectId: infrastructure-prod-92821
  settings:
    authType: public
    components:
    - alias: SD
      expireAfter: 60000
      latency:
        boostFactor: 1
        current: 150
        tolerance: 500
      name: asia.multiple.spot.multiple.public.SD.USD.basket.global
    - alias: USDT
      expireAfter: 60000
      latency:
        boostFactor: 1
        current: 150
        tolerance: 500
      name: basket.multiple.spot.multiple.public.USDT.USD.asia.global
    currency1: SD
    currency2: USDT
    customName: global
    excha

##### Vol Index

In [110]:
## TODO: add vol index 

##### Price Index

In [111]:
## TODO: add Price Index

#### Tradegenerator & Tradeexecutor (CEX)

##### Tradegenerator

In [ ]:
def cex_tradegen():
    pass